# Using `surprise`

See the documentation [here](https://surprise.readthedocs.io/en/stable/getting_started.html)!

In [ ]:
import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

## Agenda

SWBAT:

- use the `surprise` package to build recommendation engines.

In [ ]:
data = surprise.Dataset.load_builtin('ml-100k')

Now that we've downloaded the data, we can find it in a hidden directory:

In [ ]:
df = pd.read_csv('~/.surprise_data/ml-100k/ml-100k/u.data',
            sep='\t', header=None)
df = df.rename(columns={0: 'user', 1: 'item', 2: 'rating', 3: 'timestamp'})
df

## Data Exploration

In [ ]:
df['user'].nunique()

In [ ]:
df['item'].nunique()

In [ ]:
stats = df[['rating', 'timestamp']].describe()
stats

In [ ]:
print(dt.datetime.fromtimestamp(stats.loc['min', 'timestamp']))
print(dt.datetime.fromtimestamp(stats.loc['max', 'timestamp']))

In [ ]:
read = surprise.Reader('ml-100k')

In [ ]:
read.rating_scale

## Modeling

In [ ]:
train, test = surprise.model_selection.train_test_split(data, random_state=42)

In [ ]:
model = KNNBasic().fit(train)

$\hat{r}_{ui} = \frac{
    \sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{vi}}
    {\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)}$
    OR
$\hat{r}_{ui} = \frac{
    \sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}}
    {\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}$

In [ ]:
model2 = SVD().fit(train)

$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
    \lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right)$

In [ ]:
model3 = NMF().fit(train)

$\hat{r}_{ui} = q_i^Tp_u$

In [ ]:
model.get_neighbors(iid=51, k=1)

In [ ]:
conds = [df['item'] == 51, df['item'] == 65]
choices = 2*[True]

df.loc[np.select(conds, choices, default=False)].sort_values('user')

## Evaluation

In [ ]:
model.test(test)

In [ ]:
surprise.accuracy.mae(model.test(test))

In [ ]:
surprise.accuracy.mae(model2.test(test))

In [ ]:
surprise.accuracy.mae(model3.test(test))

In [ ]:
surprise.accuracy.rmse(model.test(test))

In [ ]:
surprise.accuracy.rmse(model2.test(test))

In [ ]:
surprise.accuracy.rmse(model3.test(test))